## Conversational UI Chatbot App with ChatGPT, Langchain and Streamlit

#### Here we will vuild a advanced ChatGPT conversational UI based chatbot using Langchain and Streamlit with the following features:
- Custom Landing Page
- Conversational memory
- Result streaming capabilities ( Real time output)

#### Step 1: Install App and LLM dependencies

In [1]:
!pip install langchain==0.1.12
!pip install langchain-openai==0.0.8
!pip install langchain-community==0.0.29
!pip install streamlit==1.32.2
!pip install pyngrok==7.1.5
!pip install python-dotenv # installing dotenv

#### Step 2: Load OpenAI API Credentials

In [2]:
import os
from dotenv import load_dotenv, find_dotenv

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
load_dotenv('/content/drive/MyDrive/ChatGPT Langchain Streamlit/env.txt')

True

In [5]:
import os
import openai

# Set the OpenAI API key
OPENAI_API_KEY = os.environ["OPENAI_API_KEY"]

# Set the API key for the OpenAI client
openai.api_key = OPENAI_API_KEY
client = openai.api_key = OPENAI_API_KEY
client = openai

#### Step 3: App code development

In [6]:
# Using magic command, create app.py file by writing all the code to the
# python file app.py
# We will then deploy this app.py file on the cloud server where Google colab is running

%%writefile app.py

from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_community.chat_message_histories import StreamlitChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain.callbacks.base import BaseCallbackHandler
from operator import itemgetter
import streamlit as st

# Customize initial app landing page
st.set_page_config(page_title="AI Assistant", page_icon="🤖")
st.title("Welcome I am AI Assistant 🤖")

# Manages live updates to a Streamlit app's display by appending new text tokens
# to an existing text stream and rendering the updated text in Markdown
class StreamHandler(BaseCallbackHandler):
  def __init__(self, container, initial_text=""):
    self.container = container
    self.text = initial_text

  def on_llm_new_token(self, token: str, **kwargs) -> None:
    self.text += token
    self.container.markdown(self.text)

# Load a connection to ChatGPT LLM
chatgpt = ChatOpenAI(model_name='gpt-3.5-turbo', temperature=0.1,
                     streaming=True)

# Add a basic system prompt for LLM behavior
SYS_PROMPT = """
              Act as a helpful assistant and answer questions to the best of your ability.
              Do not make up answers.
              """

# Create a prompt template for langchain to use history to answer user prompts
prompt = ChatPromptTemplate.from_messages(
  [
    ("system", SYS_PROMPT),
    MessagesPlaceholder(variable_name="history"),
    ("human", "{input}"),
  ]
)

# Create a basic llm chain
llm_chain = (
  prompt
  |
  chatgpt
)

# Store conversation history in Streamlit session state
streamlit_msg_history = StreamlitChatMessageHistory()

# Create a conversation chain
conversation_chain = RunnableWithMessageHistory(
  llm_chain,
  lambda session_id: streamlit_msg_history,  # Accesses memory
  input_messages_key="input",
  history_messages_key="history",
)

# Shows the first message when app starts
if len(streamlit_msg_history.messages) == 0:
  streamlit_msg_history.add_ai_message("How can I help you?")

# Render current messages from StreamlitChatMessageHistory
for msg in streamlit_msg_history.messages:
  st.chat_message(msg.type).write(msg.content)

# If user inputs a new prompt, display it and show the response
if user_prompt := st.chat_input():
  st.chat_message("human").write(user_prompt)
  # This is where response from the LLM is shown
  with st.chat_message("ai"):
    # Initializing an empty data stream
    stream_handler = StreamHandler(st.empty())
    config = {"configurable": {"session_id": "any"},
              "callbacks": [stream_handler]}
    # Get llm response
    response = conversation_chain.invoke({"input": user_prompt},
                                         config)







Writing app.py


#### Step 4: Start the app

In [7]:
!streamlit run app.py --server.port=8989 &>./logs.txt &

In [8]:
from pyngrok import ngrok
import yaml

# Terminate open tunnels if exist
ngrok.kill()

# Setting the authtoken
# Get your authtoken from `ngrok_credentials.yml` file
with open('/content/drive/MyDrive/ChatGPT Langchain Streamlit/ngrok_credentials.yml', 'r') as file:
    NGROK_AUTH_TOKEN = yaml.safe_load(file)
ngrok.set_auth_token(NGROK_AUTH_TOKEN['ngrok_key'])

# Open an HTTPs tunnel on port XXXX which you get from your `logs.txt` file
ngrok_tunnel = ngrok.connect(8989)
print("Streamlit App:", ngrok_tunnel.public_url)

Streamlit App: https://ff34-34-125-122-160.ngrok-free.app
